 **Happy Bus Day**


* 시도 모델
    1. machine learning (regressor)
        * 1. decision tree / 2. random forest / 3. svm
        * 4. kneighbors / 5. gaussian nb / 6. extra tree
        * 7. adaboost / 8. gradboost / 9. xgboost
        * 10. huber / 11. theil-sen / 12. polynomial

    2. deep learning 
        * DNN
            * layer : 1, 2, 3
            * dropout : 0.1, 0.3, 0.5, 0.9
            * optimizer : adam, adamW
            * activation : relu, leaky relu

    3. ensemble
        * voting regressor
        * mean score

* 시도 및 착안 아이디어
    1. correlation 활용 feature select (O)
    2. 새로운 column 생성
    3. voting regressor (X)
    4. 중간 값 이용 앙상블 (O)
    5. grid search (O)
    6. optuna (X)
    7. 반올림, 내림 (O)

# pip, install, functions

In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn import decomposition
from sklearn import datasets

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

from tqdm import tqdm

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn import model_selection

functions used for optuna (xgboost)

In [ ]:
%%capture
!pip install optuna 

In [ ]:
import optuna

import xgboost as xgb
# from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
def NMAE(true, pred):
    score = np.mean(np.abs(true - pred) / true)
    return score

# load, split and make data

## load

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

## split X_train y_train

In [ ]:
X_train = train.drop(['ID','착과량(int)'], axis=1)
y_train = train['착과량(int)']
test = test.drop(['ID'], axis=1)

* 13개 selected cols

In [ ]:
X_train_13col = X_train[['2022-09-01 새순', '2022-09-02 새순', '2022-09-03 새순', '2022-09-04 새순',
       '2022-09-05 새순', '2022-09-06 새순', '2022-09-07 새순', '2022-09-08 새순',
       '2022-09-09 새순', '2022-09-10 새순', '2022-09-11 새순', '2022-09-13 새순',
       '2022-09-16 새순']]

* 5개 selected cols

In [ ]:
#5Features_1
X_train_5col = X_train[['2022-09-01 새순', '2022-09-02 새순', '2022-09-04 새순',
       '2022-09-07 새순', '2022-09-10 새순']]

* 6개 selected cols

In [ ]:
#6Features_1

X_train_6col = X_train[['2022-09-01 새순', '2022-09-02 새순', '2022-09-04 새순', '2022-09-06 새순',
       '2022-09-07 새순', '2022-09-10 새순']]
X_train_6col

* 9개 cols

       5개 selected cols + 4개 middle value

In [ ]:
X_train_9col_day = pd.DataFrame(columns=['2022-09-01 새순', '2022-09-01 새순mid',  '2022-09-02 새순',  '2022-09-02 새순mid', '2022-09-04 새순', '2022-09-04 새순mid',
       '2022-09-07 새순','2022-09-07 새순mid', '2022-09-10 새순'])

lst = ['2022-09-01 새순', '2022-09-02 새순', '2022-09-04 새순','2022-09-07 새순', '2022-09-10 새순']
lst2 = ['2022-09-01 새순mid', '2022-09-02 새순mid', '2022-09-04 새순mid','2022-09-07 새순mid']

for i in lst:
    X_train_9col_day[i] = X_train[i]
    
for i in range( len(lst)-1 ):
    X_train_9col_day[lst2[i]] = (X_train_9col_day[lst[i]] + X_train_9col_day[lst[i+1]] )/2

# feature select
실행 시 train test split되므로 load data를 다시 실행해야 함

In [ ]:
%%capture
!pip install category_encoders

In [ ]:
from category_encoders import OneHotEncoder
encoder = OneHotEncoder(use_cat_names = True)
df1 = encoder.fit_transform(train)

In [ ]:
from sklearn.model_selection import train_test_split
X = df1[df1.columns.difference(['착과량(int)'])]
y = df1[['착과량(int)']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(df1.shape)

(1765, 2389)
(442, 2389)
(1765, 1)
(442, 1)
(2207, 2390)


In [ ]:
# target(Price)와 가장 correlated 된 features 를 k개 고르기.
## f_regresison, SelectKBest 불러오기.
from sklearn.feature_selection import f_regression, SelectKBest
## selctor 정의하기.
selector = SelectKBest(score_func=f_regression, k=5)
## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)
## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


((1765, 5), (442, 5))

In [ ]:
all_names = X_train.columns
## selector.get_support()
selected_mask = selector.get_support()
## 선택된 특성(변수)들
selected_names = all_names[selected_mask]
## 선택되지 않은 특성(변수)들
unselected_names = all_names[~selected_mask]
print('Selected names: ', selected_names)
print('Unselected names: ', unselected_names)

Selected names:  Index(['2022-09-01 새순', '2022-09-02 새순', '2022-09-04 새순', '2022-09-06 새순',
       '2022-09-07 새순'],
      dtype='object')
Unselected names:  Index(['2022-09-01 엽록소', '2022-09-02 엽록소', '2022-09-03 새순', '2022-09-03 엽록소',
       '2022-09-04 엽록소', '2022-09-05 새순', '2022-09-05 엽록소', '2022-09-06 엽록소',
       '2022-09-07 엽록소', '2022-09-08 새순',
       ...
       'ID_TRAIN_2201', 'ID_TRAIN_2202', 'ID_TRAIN_2203', 'ID_TRAIN_2204',
       'ID_TRAIN_2205', 'ID_TRAIN_2206', '수고(m)', '수관폭1(min)', '수관폭2(max)',
       '수관폭평균'],
      dtype='object', length=2384)


# model define

In [ ]:
dtr = DecisionTreeRegressor()
lgr = LogisticRegression()
rfr = RandomForestRegressor()
svr = SVR()
knnr = KNeighborsRegressor()

gnb = GaussianNB()
extree = ExtraTreesRegressor()
adaboost = AdaBoostRegressor()
mlp = MLPRegressor()
gradboost = GradientBoostingRegressor()

huber = HuberRegressor()
theilsen = TheilSenRegressor()
p_model2 = Pipeline([('poly',PolynomialFeatures(degree=2, include_bias=False)),
                    ('linear',LinearRegression())])


In [ ]:
# example
dtr = DecisionTreeRegressor()

dtr.fit(X_train, y_train)

# grid search

In [ ]:
###### choose model category above defined #######

estimator = RandomForestRegressor()

param_grid = {'criterion':['mae','mse'], 'max_depth':[5]} 
grid = GridSearchCV(estimator, param_grid=param_grid) 
grid.fit(X_train_13col, y_train)

In [ ]:
estimator = RandomForestRegressor()
estimator.set_params(**grid.best_params_)

estimator.fit(X_train, y_train)

In [ ]:
pred = estimator.predict(test)

# optuna (xgboost)

In [ ]:
def objective(trial, data=X_train, target=y_train):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'loss_function': 'NMAE',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17])
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    nmae = NMAE(test_y, preds)
    # mae = mean_absolute_error(test_y, preds) # ,squared=False)
    
    return nmae

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # n_trials 가 epoch

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
param = {
    'loss_function': 'NMAE',
    'learning_rate': 0.008,
    'n_estimators': 10000,
    'max_depth': 17
}
xgboost = xgb.XGBRegressor(**param)  

# voting regressor

In [ ]:
voting = VotingRegressor([('extra tree', extree),
                        ('adabst', adaboost),
                        ('gradbst', gradboost),
                        ('knnr', knnr),
                        ('Decision Tree', decitree),
                        ('Random Forest', rndforest)])

pred = voting.fit(X_train, y_train).predict(test)

# mean ensemble

In [ ]:
A = pd.read_csv("A.csv")
B = pd.read_csv("B.csv")
C = pd.read_csv("C.csv")
D = pd.read_csv("D.csv")

In [ ]:
###### 작성 ######
models = [A, B, C, D]
##################

dic = {'ID' :[], '착과량(int)' :[]}

for i in range(len(R['ID'])) :
    tmp = []
    dic['ID'].append(R['ID'][i])
    for esb in models:
        tmp.append( esb['착과량(int)'][i] )

    dic['착과량(int)'].append(np.mean(tmp))

result = pd.DataFrame(dic)
# result.to_csv('my.csv',index = False)

# submission

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
###########    pred 수정      ############

# sample_submission['착과량(int)'] = pred                   
sample_submission['착과량(int)'] = result['착과량(int)']    # mean ensemble

##########################################


# sample_submission['착과량(int)'] = sample_submission['착과량(int)'].map(lambda x: round(x)) # 반올림
# sample_submission['착과량(int)'] = sample_submission['착과량(int)'].astype(int)  # 내림


sample_submission.to_csv('./result.csv', index=False)